In [4]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import mysql.connector
import ssl

# inisialisasi array untuk menampilkan hasil
id_detail = []
keterangan_lowongan = []
deskripsi_lowongan = []
skill_lowongan = []
benefit_lowongan = []
stem_detail = []

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()
add = 0

# dari sini
cursor.execute("SELECT id_lowongan, stem_detail, link_lowongan FROM lowongan WHERE link_lowongan LIKE 'https://glints.com%'")
raw_lowongan = cursor.fetchall()
for row in raw_lowongan:
    id_lowongan = row[0]
    
    if row[1] == "":
        link = row[2]
        r = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        gcontext = ssl.SSLContext()  # Only for gangstars
        response = urlopen(r, context=gcontext).read()
        soup = BeautifulSoup(response, "lxml")

        raw_detail = soup.find_all("div","Opportunitysc__Container-sc-1gsvee3-2 gPhXzZ")
        for p in raw_detail:
            # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
            try:
                raw_keterangan = soup.find_all("div", "TopFoldsc__JobOverViewInfo-kklg8i-9 eWKiJR")
                keterangan = ""
                for q in raw_keterangan:
                    keterangan = q.get_text() + ". " + keterangan
            except:
                keterangan = "-"
            
            # menggunakan try except karena ada beberapa deskripsi yang NoneType
            try:
                deskripsi = p.find("div", "Opportunitysc__JobDescriptionContainer-sc-1gsvee3-6 gBmrdW" ).get_text(separator=" ").replace("Informasi Penting Pastikan perusahaan yang kamu lamar resmi dengan memeriksa website dan lowongan kerja mereka. Read Less ", "").replace("Deskripsi pekerjaan ", "")
            except:
                deskripsi = "Eror: deskripsi Nonetype"

            try:
                skill = p.find("div", "Opportunitysc__SkillsContainer-sc-1gsvee3-7 hzDxPi").get_text(separator=" ")
            except:
                skill = '-'
            try:
                benefit = p.find("div", "Opportunitysc__BenefitsContainer-sc-1gsvee3-8 jvOXrL").get_text(separator=" ")
            except:
                benefit = '-'

            raw_stem = deskripsi + " " + skill + " " + benefit
            # define punctuation
            punctuations = '''!()-[]{};:=+`'",<>./|\?@#$%^&*_~'''

            # remove punctuation from the string
            no_punct = ""
            for char in raw_stem:
                if char not in punctuations:
                    no_punct = no_punct + char
                else:
                    no_punct = no_punct + " "

            # display the unpunctuated stringZZz
            raw_stem = no_punct.lower()

            # create stemmer
            # nltk
            ps = PorterStemmer()
            # sastrawi
            factory = StemmerFactory()
            stemmer = factory.create_stemmer()

            # stemming process
            # nltk
            words = word_tokenize(raw_stem)
            nltk_stemmer = ""
            for w in words:
                nltk_stemmer = nltk_stemmer + " " + ps.stem(w)
            # sastrawi
            output_stemmer = stemmer.stem(nltk_stemmer)

            cursor.execute("UPDATE lowongan SET stem_detail = '"+ output_stemmer +"' WHERE link_lowongan = '"+ link + "'")
            db.commit()
            add = add + 1

            id_detail.append(id_lowongan)
            keterangan_lowongan.append(keterangan)
            deskripsi_lowongan.append(deskripsi)
            skill_lowongan.append(skill)
            benefit_lowongan.append(benefit)
            stem_detail.append(output_stemmer)

print("Data has been added: " + str(add))
detail_dict ={'id':id_detail, 'keterangan':keterangan_lowongan, 'deskripsi':deskripsi_lowongan, 'skill':skill_lowongan, 'benefit':benefit_lowongan, 'stem':stem_detail}
df = pd.DataFrame(detail_dict, columns = ['id', 'keterangan', 'deskripsi', 'skill', 'benefit', 'stem'])

df.sort_values('id', ascending=True)

Data has been added: 0


,id,keterangan,deskripsi,skill,benefit,stem
